In [2]:
import keras as k
import pandas as pd

Using TensorFlow backend.


In [3]:
df = pd.read_csv("./train.csv")

In [5]:
"""
Index: A unique value that labels each row in the data set

Day: The day of the year

Market: A label that indicates which exchange the instrument is traded on

Stock: A unique label for each instrument

x0...x3E: Predictors that relate to the observed behaviour of the instrument on
    the day in question. 
    The features labelled 'x3A', 'x3B', etc. are strongly related and might be expected to be highly correlated.
    
x4...x6: Predictors that describe the ‘typical’ behaviour 
    that we would expect of the instrument on that day.
    
Weight: an importance weight we have generated, used in the wMSE scoring function.
    Like y, this isn't present in the test data.
    
THOUGHTS:
    - separate into separate data frames for each market. use df.groupby
    - train each model separately
    
    
"""
df.head()



,Index,Market,Day,Stock,x0,x1,x2,x3A,x3B,x3C,x3D,x3E,x4,x5,x6,y,Weight
0,0,1,1,363,0.002696,0.530040,0.400810,1.300048e-06,4.639533e-06,1.073594e-05,2.180516e-05,4.416943e-05,0.002212,6.214737e-07,200.000000,-0.000196,1.533780
1,1,1,1,1223,0.081093,1.366218,1.325157,1.656861e-05,3.024668e-05,5.261872e-05,4.193762e-05,4.193762e-05,0.010932,3.414868e-05,300.000000,-0.003798,0.843604
2,2,1,1,1372,0.009561,0.464871,0.328008,6.500872e-08,2.323204e-07,5.182686e-07,1.080834e-06,2.030442e-06,0.000306,4.079460e-08,159.692076,-0.000003,29.824024
3,3,1,1,1942,0.045300,0.608222,0.456662,1.928694e-08,6.495275e-08,1.338509e-07,2.644214e-07,6.980798e-07,0.000205,9.502066e-09,212.160378,-0.000008,100.388668
4,4,1,1,2712,0.001605,0.476718,0.205069,6.759866e-07,2.638783e-06,6.424519e-06,1.189923e-05,2.064450e-05,0.001525,3.428403e-07,153.724351,0.000065,5.950714


In [9]:
stocks = df.groupby('Stock').nunique()
stocks.shape

(3022, 17)

In [10]:
df.shape

(623817, 17)

In [13]:
stocks.tail()

,Index,Market,Day,Stock,x0,x1,x2,x3A,x3B,x3C,x3D,x3E,x4,x5,x6,y,Weight
Stock,,,,,,,,,,,,,,,,,
3018,170,1,170,1,170,170,170,170,170,170,170,170,169,170,75,170,160
3019,247,1,247,1,247,247,247,247,247,247,247,247,247,247,114,247,242
3020,249,1,249,1,249,249,249,249,249,249,249,249,241,249,41,249,220
3021,255,1,255,1,255,255,255,255,255,255,255,255,248,255,208,255,225
3022,247,1,247,1,247,247,247,247,247,247,247,247,240,247,95,247,234


In [14]:
markets = df.groupby('Market').nunique()
markets.shape

(4, 17)

In [43]:
m = df.groupby('Market')['Day']

In [44]:
m.nunique()

Market
1    249
2    251
3    248
4    256
Name: Day, dtype: int64

In [36]:
markets = df.groupby('Market')


In [59]:
markets = {
    1: df[df['Market'] == 1],
    2: df[df['Market'] == 2],
    3: df[df['Market'] == 3],
    4: df[df['Market'] == 4],

}


In [61]:
markets[1].head()


,Index,Market,Day,Stock,x0,x1,x2,x3A,x3B,x3C,x3D,x3E,x4,x5,x6,y,Weight
0,0,1,1,363,0.002696,0.530040,0.400810,1.300048e-06,4.639533e-06,1.073594e-05,2.180516e-05,4.416943e-05,0.002212,6.214737e-07,200.000000,-0.000196,1.533780
1,1,1,1,1223,0.081093,1.366218,1.325157,1.656861e-05,3.024668e-05,5.261872e-05,4.193762e-05,4.193762e-05,0.010932,3.414868e-05,300.000000,-0.003798,0.843604
2,2,1,1,1372,0.009561,0.464871,0.328008,6.500872e-08,2.323204e-07,5.182686e-07,1.080834e-06,2.030442e-06,0.000306,4.079460e-08,159.692076,-0.000003,29.824024
3,3,1,1,1942,0.045300,0.608222,0.456662,1.928694e-08,6.495275e-08,1.338509e-07,2.644214e-07,6.980798e-07,0.000205,9.502066e-09,212.160378,-0.000008,100.388668
4,4,1,1,2712,0.001605,0.476718,0.205069,6.759866e-07,2.638783e-06,6.424519e-06,1.189923e-05,2.064450e-05,0.001525,3.428403e-07,153.724351,0.000065,5.950714
